## **02 - Feature Engineering (FE)**

**Source**

This notebook starts from the 

**Goals**
- Transform the canonical EDA dataset into a model-ready feature matrix without redefining the prediction problem
- Preserve a strict separation of concerns between:
    - EDA (problem definition and data semantics)
    - Feature engineering (input representation)
    - Modeling (learning and evaluation)
- Ensure all feature transformations are:
    - Deterministic
    - Reproducible
    - Auditable
- Minimize the risk of data leakage by:
    - Using only information available at prediction time
    - Applying all target-independent transformations
- Maintain model-agnostic feature representations that can be reused across:
    - Linear models
    - Tree-based models
    - Ensemble methods
- Prioritize interpretability and transparency, especially in a credit risk context
- Enable a clean handoff to a production-grade feature build script driven by a single feature specification
- Support iterative experimentation through explicit versioning, rather than ad-hoc feature changes

**To-Do Checklist** 

1. Inputs & Data Contracts
- [x] Load cleaned dataset produced by EDA
- [x] Load eda_summary.json
- [x] Verify EDA summary contains target, drop columns, numeric, categorical, and datetime columns
- [x] Validate dataset schema matches EDA expectations
- [x] Fail early if schema mismatch is detected

2. Target Variable Handling
- [x] Confirm target column exists
- [x] Verify target encoding is binary
- [x] Validate class distribution
- [x] Freeze target definition
- [x] Exclude target from feature transforms

3. Feature Exclusion Logic
- [x] Drop explicitly excluded columns
- [x] Drop constant and low-variance columns
- [x] Drop identifier columns
- [x] Verify dropped columns are not used downstream
- [x] Log total columns removed

4. Feature Type Validation
- [x] Validate numeric dtypes and ranges
- [x] Validate categorical cardinality
- [x] Validate datetime parsing
- [x] Assert column type disjointness

5. Datetime Feature Engineering
- [x] Convert datetime columns explicitly
- [x] Derive year, month, quarter features
- [x] Validate temporal logic
- [x] Drop raw datetime columns
- [x] Document decisions

6. Categorical Feature Engineering
- [x] Compute cardinality
- [x] Assign encoding strategy by tier
- [x] Consolidate rare categories
- [x] Handle missing categories
- [x] Document encoding per feature

7. Numerical Feature Engineering
- [x] Inspect distributions
- [x] Identify skew

8. Candidate Engineered Features
- [x] Enumerate candidate engineered features
- [x] Identify source columns for each candidate
- [x] Confirm availability at prediction time
- [x] Assess leakage risk
- [x] Assess interpretability and stability
- [x] Decision recorded for each:
    - include in v1
- [x] Rationale documented

9. Feature Interactions (Optional)
- [x] Identify justified interactions
- [x] Avoid combinatorial explosion
- [x] Validate inference availability
- [x] Document rationale

10. Missing Value Strategy (Design Only)
- Identify missingness
- Define numeric and categorical strategies
- Do not fit imputers here
- Defer fitting to build script

11. Feature Leakage Checks
- Ensure no post-outcome features
- Validate prediction-time availability
- Check datetime cutoffs
- Document assumptions

12. Feature Specification Output
- Create feature spec
- Include version, features, drops, transforms
- Store under configs/
- Treat as single source of truth

---

Definition of Done  
A model can be trained without this notebook using only the feature spec and the feature build script.



# 1. Inputs & Data Contracts

We will:

- Load cleaned dataset produced by EDA
- Load eda_summary.json
- Verify EDA summary contains target, drop columns, numeric, categorical, and datetime columns
- Validate dataset schema matches EDA expectations
- Fail early if schema mismatch is detected

In [24]:
import json
from pathlib import Path

import pandas as pd

# -----------------------------
# Paths
# -----------------------------
DATA_DIR = Path("../data/_artifacts_preview")

EDA_DATA_PATH = DATA_DIR / "eda_cleaned.parquet"
EDA_SCHEMA_PATH = DATA_DIR / "eda_cleaned_schema.json"
EDA_SUMMARY_PATH = DATA_DIR / "eda_summary.json"

CONFIGS_DIR = Path("configs")
CONFIGS_DIR.mkdir(parents=True, exist_ok=True)

# Where we'll eventually write the feature spec
target_metadata_PATH = CONFIGS_DIR / "target_metadata_v1.json"

# -----------------------------
# Load artifacts
# -----------------------------
for p in [EDA_DATA_PATH, EDA_SCHEMA_PATH, EDA_SUMMARY_PATH]:
    if not p.exists():
        raise FileNotFoundError(f"Missing required artifact: {p}")

df = pd.read_parquet(EDA_DATA_PATH)

with open(EDA_SCHEMA_PATH, "r") as f:
    eda_schema = json.load(f)

with open(EDA_SUMMARY_PATH, "r") as f:
    eda_summary = json.load(f)

print("Loaded:")
print(f"- dataset: {EDA_DATA_PATH} | shape={df.shape}")
print(f"- schema:  {EDA_SCHEMA_PATH}")
print(f"- summary: {EDA_SUMMARY_PATH}")

# -----------------------------
# Validate EDA summary contract keys
# -----------------------------
required_summary_keys = [
    "target_variable",
    "numerical_cols",
    "categorical_cols",
    "datetime_cols",
    "cols_to_drop",
    "constant_cols",
    "low_variance_cols",
]

missing_keys = [k for k in required_summary_keys if k not in eda_summary]
if missing_keys:
    raise KeyError(f"eda_summary.json missing required keys: {missing_keys}")

TARGET = eda_summary["target_variable"]

# -----------------------------
# Schema alignment checks (fail fast)
# -----------------------------
if TARGET not in df.columns:
    raise KeyError(f"Target column '{TARGET}' not found in EDA dataset columns")

num_cols = set(eda_summary["numerical_cols"])
cat_cols = set(eda_summary["categorical_cols"])
dt_cols = set(eda_summary["datetime_cols"])

columns_to_drop = set(eda_summary.get("cols_to_drop", []))
constant_cols = set(eda_summary.get("constant_cols", []))
low_variance_cols = set(eda_summary.get("low_variance_cols", []))

cols_to_drop = columns_to_drop | constant_cols | low_variance_cols

# The EDA dataset should contain only family columns + target
expected_cols = num_cols | cat_cols | dt_cols | {TARGET}

# ensure no unexpected columns
expected_cols -= cols_to_drop

actual_cols = set(df.columns)

missing_cols = sorted(list(expected_cols - actual_cols))
extra_cols = sorted(list(actual_cols - expected_cols))

if missing_cols:
    raise ValueError(f"EDA dataset missing expected columns: {missing_cols}")

if extra_cols:
    raise ValueError(f"EDA dataset contains unexpected columns: {extra_cols}")

print("EDA contract check passed:")
print(f"- target: {TARGET}")
print(f"- numerical: {len(num_cols)} | categorical: {len(cat_cols)} | datetime: {len(dt_cols)}")

# -----------------------------
# Initialize design-only feature matrix handles
# -----------------------------
# We will not mutate X in this notebook; transformations belong in src/features.
X = df.drop(columns=[TARGET])
y = df[TARGET]

# -----------------------------
# Initialize feature spec skeleton (in-memory only for now)
# -----------------------------
feature_spec = {
    "version": "v1",
    "source": {
        "dataset": str(EDA_DATA_PATH),
        "schema": str(EDA_SCHEMA_PATH),
        "eda_summary": str(EDA_SUMMARY_PATH),
    },
    "target": {
        "name": TARGET,
        # FE freezes shape guarantees, not semantic definition
        "dtype": str(y.dtype),
        "allowed_values": [0, 1],
        "missing_allowed": False,
        "positive_label": 1,
        "positive_rate_at_freeze": float(y.mean()),
        "definition_source": "EDA",
    },
    "features": {
        # Step 3 will populate exclusions
        # Step 5 will populate datetime
        # Step 6 will populate categorical + categorical_defaults
        # Step 7 will populate numerical + numerical_transform_rules
    },
    "freeze": {
        # filled at the end when writing the final spec
        "created_at": None,
        "spec_hash_md5": None,
        "target_hash_md5": None,
    },
}

print("Initialized target_metadata skeleton (not written yet).")
print(f"Will write spec to: {target_metadata_PATH}")

Loaded:
- dataset: ../data/_artifacts_preview/eda_cleaned.parquet | shape=(2252232, 78)
- schema:  ../data/_artifacts_preview/eda_cleaned_schema.json
- summary: ../data/_artifacts_preview/eda_summary.json
EDA contract check passed:
- target: default
- numerical: 113 | categorical: 37 | datetime: 1
Initialized target_metadata skeleton (not written yet).
Will write spec to: configs/target_metadata_v1.json


# 2. Target Variable Handling

In [25]:
import hashlib
import json

# confirm target column exists
assert TARGET in df.columns, f"Target column '{TARGET}' not found in dataset"

# confirm target column is binary
unique_target_values = y.unique()
if len(unique_target_values) != 2:
    raise ValueError(
        f"Target column '{TARGET}' is not binary. Unique values found: {unique_target_values}"
    )

# validate class distribution
print("Target distribution:")
print(y.value_counts(normalize=True))

# assert existence and uniqueness of target column
assert TARGET in df.columns, f"Target column '{TARGET}' not found in dataset"

# assert no missing values in target column
assert df[TARGET].notna().all(), f"Target column '{TARGET}' contains missing values"

# assert encoding shape
unique_values = set(df[TARGET].unique())
unique_values = {int(val) for val in unique_values}  # ensure values are integers
assert unique_values.issubset({0, 1}), (
    f"Target column '{TARGET}' contains unexpected values: {unique_values}"
)

# record feature spec target and certain metadata
feature_spec = {
    "version": "v1",
    "source": {
        "dataset": "eda_cleaned.parquet",
        "schema": "eda_cleaned_schema.json",
        "eda_summary": "eda_summary.json",
    },
    "target": {
        "name": TARGET,
        "dtype": str(df[TARGET].dtype),
        "unique_values": sorted(unique_values),
        "missing_allowed": False,
        "positive_rate": float(df[TARGET].mean()),
        "source": "eda_cleaned.parquet",
    },
    "features": {},
    "encoding_fit_rules": {
        "target_mean": {
            "fit_on": "train_only",
            "cv_folds": 5,
            "smoothing": "auto",
            "handle_unknown": "prior",
            "handle_missing": "prior",
        }
    },
    "output": {"feature_ordering": "stable", "sort_columns": True},
}

# exclude target from features
EXCLUDED_FROM_FEATURES = {TARGET}

assert TARGET not in set(eda_summary["numerical_cols"]), (
    f"Target column '{TARGET}' should not be in numerical columns"
)
assert TARGET not in set(eda_summary["categorical_cols"]), (
    f"Target column '{TARGET}' should not be in categorical columns"
)
assert TARGET not in set(eda_summary["datetime_cols"]), (
    f"Target column '{TARGET}' should not be in datetime columns"
)

# hash-based freeze of target definition

feature_spec["freeze"] = {
    "created_at": pd.Timestamp.now().isoformat(),
    "target_hash_md5": hashlib.md5(df[TARGET].to_numpy().tobytes()).hexdigest(),
    "spec_hash_md5": hashlib.md5(
        json.dumps(feature_spec["target"], sort_keys=True).encode()
    ).hexdigest(),
}

# create feature specification in /configs directory (not shown here)
ARTIFACTS_DIR = Path("../data/_artifacts_preview")
FEATURE_SPEC_PATH = ARTIFACTS_DIR / "feature_spec_v1.json"

Target distribution:
default
0    0.868941
1    0.131059
Name: proportion, dtype: float64


# 3. Feature Exclusion Logic

In [26]:
# ============================================================
# 3. Feature Exclusion Logic (FIXED: deterministic, no heuristics)
# ============================================================

from collections import defaultdict

TARGET = eda_summary["target_variable"]

# EDA-driven exclusion sources (contract)
cols_to_drop = set(eda_summary.get("cols_to_drop", []))
constant_cols = set(eda_summary.get("constant_cols", []))
low_variance_cols = set(eda_summary.get("low_variance_cols", []))

# OPTIONAL: explicit identifiers (only if you choose to maintain them)
# Put this list into the spec once you’ve confirmed them.
explicit_identifier_cols = set(feature_spec.get("exclusions", {}).get("identifier_cols", []))

df_cols = set(df.columns)

# Build exclusion sets by reason (only columns that actually exist, never target here)
by_reason = defaultdict(list)


def keep_if_exists(cols):
    return sorted([c for c in cols if c in df_cols and c != TARGET])


by_reason["cols_to_drop"] = keep_if_exists(cols_to_drop)
by_reason["constant_cols"] = keep_if_exists(constant_cols)
by_reason["low_variance_cols"] = keep_if_exists(low_variance_cols)
by_reason["identifier_cols"] = keep_if_exists(explicit_identifier_cols)

# Unified excluded feature columns (not including target)
excluded_feature_cols = sorted(set().union(*[set(v) for v in by_reason.values()]))

# Sanity: target must not be in excluded_feature_cols (target exclusion is separate)
if TARGET in excluded_feature_cols:
    raise ValueError(
        f"Target '{TARGET}' appears in excluded_feature_cols (should be excluded separately)."
    )

# Defensive: show if any excluded cols appear in feature families (not always wrong, but must be known)
num_cols = set(eda_summary.get("numerical_cols", []))
cat_cols = set(eda_summary.get("categorical_cols", []))
dt_cols = set(eda_summary.get("datetime_cols", []))
family_cols = num_cols | cat_cols | dt_cols

overlap_with_families = sorted(list(set(excluded_feature_cols) & family_cols))
if overlap_with_families:
    print(
        "NOTE: Some excluded columns also appear in EDA feature families (review intentionality):"
    )
    print(overlap_with_families[:30], "..." if len(overlap_with_families) > 30 else "")

# Write exclusions into feature_spec (single source of truth for downstream)
feature_spec["exclusions"] = {
    "target": TARGET,
    "excluded_feature_cols": excluded_feature_cols,
    "by_reason": dict(by_reason),
    # Keep identifiers explicit—do NOT infer with heuristics
    "identifier_cols": sorted(list(explicit_identifier_cols)),
    "notes": (
        "Exclusions are derived from EDA contract (cols_to_drop, constant_cols, low_variance_cols) "
        "plus optional explicit identifier_cols. No heuristic-based dropping is used."
    ),
}

# Reviewer-facing summary
rows = []
for reason, cols in by_reason.items():
    rows.append({"reason": reason, "n_cols": len(cols), "example_cols": cols[:10]})

excl_df = pd.DataFrame(rows).sort_values("n_cols", ascending=False)
print(f"Excluded feature columns (not including target): {len(excluded_feature_cols)}")
display(excl_df)

Excluded feature columns (not including target): 0


,reason,n_cols,example_cols
0,cols_to_drop,0,[]
1,constant_cols,0,[]
2,low_variance_cols,0,[]
3,identifier_cols,0,[]


# 4. Feature Type Validation

In [27]:
import pandas as pd

TARGET = eda_summary["target_variable"]

# Feature families from EDA contract
num_cols = list(eda_summary.get("numerical_cols", []))
cat_cols = list(eda_summary.get("categorical_cols", []))
dt_cols = list(eda_summary.get("datetime_cols", []))

# Columns excluded by design (from Step 3)
excluded_feature_cols = set(feature_spec.get("exclusions", {}).get("excluded_feature_cols", []))

# Dataset columns available
df_cols = set(df.columns)

# --- 1) Disjointness checks (contract-level)
overlap_nc = set(num_cols) & set(cat_cols)
overlap_nd = set(num_cols) & set(dt_cols)
overlap_cd = set(cat_cols) & set(dt_cols)

if overlap_nc or overlap_nd or overlap_cd:
    raise ValueError(
        "Feature family overlap detected.\n"
        f"num∩cat: {sorted(overlap_nc)}\n"
        f"num∩dt:  {sorted(overlap_nd)}\n"
        f"cat∩dt:  {sorted(overlap_cd)}"
    )

# --- 2) Presence checks (dataset-level)
# Validate declared families against actual dataframe columns
missing_num = [c for c in num_cols if c not in df_cols]
missing_cat = [c for c in cat_cols if c not in df_cols]
missing_dt = [c for c in dt_cols if c not in df_cols]

# If declared columns are missing, warn and drop them from the working lists
if missing_num or missing_cat or missing_dt:
    print(
        "WARNING: Some EDA-declared feature columns are missing from the dataset "
        "and will be ignored.\n"
        f"missing numerical: {missing_num}\n"
        f"missing categorical: {missing_cat}\n"
        f"missing datetime: {missing_dt}"
    )
    num_cols = [c for c in num_cols if c in df_cols]
    cat_cols = [c for c in cat_cols if c in df_cols]
    dt_cols = [c for c in dt_cols if c in df_cols]

# --- 3) Ensure target is not part of feature families
if TARGET in set(num_cols) | set(cat_cols) | set(dt_cols):
    raise ValueError(f"Target column '{TARGET}' is incorrectly included in feature families.")

# --- 4) Ensure exclusions do not accidentally remove the whole feature set
family_cols = set(num_cols) | set(cat_cols) | set(dt_cols)
remaining_feature_cols = sorted(list(family_cols - excluded_feature_cols))

if len(remaining_feature_cols) == 0:
    raise ValueError("After exclusions, there are no remaining feature columns.")

# --- 5) Dtype sanity checks (lightweight, non-mutating)
# Note: We do NOT coerce types here; we only validate expectations.
dtype_rows = []

# Numeric expectations
for c in num_cols:
    s = df[c]
    dtype_rows.append(
        {
            "col": c,
            "family": "numerical",
            "dtype": str(s.dtype),
            "missing_rate": float(s.isna().mean()),
            "example_values": s.dropna().head(3).tolist(),
        }
    )

# Categorical expectations
for c in cat_cols:
    s = df[c]
    dtype_rows.append(
        {
            "col": c,
            "family": "categorical",
            "dtype": str(s.dtype),
            "missing_rate": float(s.isna().mean()),
            "example_values": s.dropna().astype(str).head(3).tolist(),
        }
    )

# Datetime expectations (parse for validation only)
for c in dt_cols:
    parsed = pd.to_datetime(df[c], errors="coerce")
    dtype_rows.append(
        {
            "col": c,
            "family": "datetime",
            "dtype": str(df[c].dtype),
            "missing_rate": float(parsed.isna().mean()),
            "example_values": parsed.dropna().head(3).astype(str).tolist(),
        }
    )

dtype_df = pd.DataFrame(dtype_rows)

# Basic flags
# Numeric columns should not be mostly non-numeric
numeric_nonnull = []
for c in num_cols:
    coerced = pd.to_numeric(df[c], errors="coerce")
    numeric_nonnull.append((c, float(coerced.notna().mean())))

bad_numeric = [c for c, frac in numeric_nonnull if frac < 0.95]  # threshold is adjustable
if bad_numeric:
    print("WARNING: Some numerical columns appear to contain substantial non-numeric values:")
    print(bad_numeric)

# Categorical cardinality summary (diagnostic)
cat_cardinality = {c: int(df[c].nunique(dropna=False)) for c in cat_cols}
high_card_cols = [c for c, k in cat_cardinality.items() if k > 1000]
if high_card_cols:
    print("NOTE: High-cardinality categorical columns detected (may require special handling):")
    print(high_card_cols)

print("Feature family validation summary:")
display(dtype_df.head(25))

# --- 6) Write a small validation record into the feature spec (optional but useful)
feature_spec["validation"] = {
    "families": {
        "numerical_count": len(num_cols),
        "categorical_count": len(cat_cols),
        "datetime_count": len(dt_cols),
        "remaining_feature_count_after_exclusions": len(remaining_feature_cols),
    },
    "overlaps": {
        "num_cat": sorted(list(overlap_nc)),
        "num_dt": sorted(list(overlap_nd)),
        "cat_dt": sorted(list(overlap_cd)),
    },
    "high_cardinality_categoricals": high_card_cols,
    "numeric_columns_low_numeric_parse_rate": bad_numeric,
}

missing numerical: ['member_id', 'funded_amnt_inv', 'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record', 'revol_util', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_amnt', 'last_fico_range_high', 'last_fico_range_low', 'collections_12_mths_ex_med', 'mths_since_last_major_derog', 'policy_code', 'acc_now_delinq', 'mths_since_rcnt_il', 'inq_last_12m', 'chargeoff_within_12_mths', 'delinq_amnt', 'mths_since_recent_bc', 'mths_since_recent_bc_dlq', 'mths_since_recent_inq', 'mths_since_recent_revol_delinq', 'num_accts_ever_120_pd', 'num_tl_30dpd', 'sec_app_inq_last_6mths', 'sec_app_revol_util', 'sec_app_collections_12_mths_ex_med', 'sec_app_mths_since_last_major_derog', 'deferral_term', 'hardship_amount', 'hardship_length', 'hardship_dpd', 'orig_projected_additional_accrued_interest', 'hardship_payoff_balance_amount', 'hardship_last_payment_amount', 

,col,family,dtype,missing_rate,example_values
0,loan_amnt,numerical,float64,0.000000,"[3600.0, 24700.0, 20000.0]"
1,funded_amnt,numerical,float64,0.000000,"[3600.0, 24700.0, 20000.0]"
2,int_rate,numerical,float64,0.000000,"[13.99, 11.99, 10.78]"
3,installment,numerical,float64,0.000000,"[123.03, 820.28, 432.66]"
4,annual_inc,numerical,float64,0.000002,"[55000.0, 65000.0, 63000.0]"
5,dti,numerical,float64,0.000756,"[5.91, 16.06, 10.78]"
6,delinq_2yrs,numerical,float64,0.000013,"[0.0, 1.0, 0.0]"
7,fico_range_low,numerical,float64,0.000000,"[675.0, 715.0, 695.0]"
8,fico_range_high,numerical,float64,0.000000,"[679.0, 719.0, 699.0]"
9,open_acc,numerical,float64,0.000013,"[7.0, 22.0, 6.0]"


# 5. Datetime Feature Engineering

In [28]:
import pandas as pd

dt_cols = eda_summary["datetime_cols"]
dt_cols_present = [c for c in dt_cols if c in X.columns]

# Ensure spec structures exist
feature_spec.setdefault("features", {}).setdefault("datetime", {})

# Define a deterministic datetime policy (spec-controlled)
# You can keep timezone=None for naive datetimes (typical for LendingClub)
DATETIME_DEFAULTS = feature_spec.get("datetime_defaults", {"timezone": None, "errors": "coerce"})
feature_spec["datetime_defaults"] = DATETIME_DEFAULTS
# Define what you will derive for each datetime column
# Keep this conservative for v1
DERIVE_COMPONENTS = feature_spec.get(
    "datetime_strategy_rules", {"derive": ["year", "month", "quarter"], "drop_raw": True}
)
feature_spec["datetime_strategy_rules"] = DERIVE_COMPONENTS
datetime_summary_rows = []

for col in dt_cols_present:
    # Parse for validation only (do not store back into X)
    parsed = pd.to_datetime(X[col], errors=DATETIME_DEFAULTS["errors"])

    missing_rate = float(parsed.isna().mean())

    # Basic sanity checks / diagnostics
    min_dt = parsed.min()
    max_dt = parsed.max()

    # Create derived feature names deterministically
    derived = []
    if "year" in DERIVE_COMPONENTS["derive"]:
        derived.append(f"{col}_year")
    if "month" in DERIVE_COMPONENTS["derive"]:
        derived.append(f"{col}_month")
    if "quarter" in DERIVE_COMPONENTS["derive"]:
        derived.append(f"{col}_quarter")

    # Write per-column datetime spec
    feature_spec["features"]["datetime"][col] = {
        "derived_features": derived,
        "drop_raw": bool(DERIVE_COMPONENTS["drop_raw"]),
        "timezone": DATETIME_DEFAULTS["timezone"],
        "missing_rate": missing_rate,
    }

    datetime_summary_rows.append(
        {
            "datetime_col": col,
            "derived_features": derived,
            "drop_raw": bool(DERIVE_COMPONENTS["drop_raw"]),
            "missing_rate": missing_rate,
            "min": None if pd.isna(min_dt) else str(min_dt),
            "max": None if pd.isna(max_dt) else str(max_dt),
        }
    )

dt_summary_df = pd.DataFrame(datetime_summary_rows)

print("Datetime feature plan (design-only):")
display(dt_summary_df)

Datetime feature plan (design-only):


,datetime_col,derived_features,drop_raw,missing_rate,min,max
0,issue_d,"[issue_d_year, issue_d_month, issue_d_quarter]",True,0.0,2007-06-01 00:00:00,2018-12-01 00:00:00


# 6. Categorical Feature Engineering

In [29]:
import numpy as np

# --- Inputs ---
cat_cols = eda_summary["categorical_cols"]
cat_cols_present = [c for c in cat_cols if c in X.columns]

# --- Use defaults from spec (or define here if building spec in notebook) ---
cat_defaults = feature_spec["features"].get("categorical_defaults", {})
MISSING_TOKEN = cat_defaults.get("missing_token", "__MISSING__")
RARE_TOKEN = cat_defaults.get("rare_token", "__OTHER__")
UNKNOWN_TOKEN = cat_defaults.get("unknown_token", "__OTHER__")

rare_threshold = cat_defaults.get("rare_threshold", {})
MIN_COUNT = int(rare_threshold.get("min_count", 100))
MIN_FRACTION = float(rare_threshold.get("min_fraction", 0.005))

# Cardinality tiers (record in spec so build script doesn't hardcode)
ONE_HOT_MAX = int(feature_spec.get("categorical_strategy_rules", {}).get("one_hot_max", 10))
TARGET_MEAN_MAX = int(
    feature_spec.get("categorical_strategy_rules", {}).get("target_mean_max", 100)
)

# Save/ensure the strategy rules exist in spec
feature_spec["categorical_strategy_rules"] = {
    "one_hot_max": ONE_HOT_MAX,
    "target_mean_max": TARGET_MEAN_MAX,
    "fallback": "count_frequency",
}


# --- Helpers ---
def normalize_missing_for_counts(s: pd.Series) -> pd.Series:
    """
    Normalize missingness for *analysis* only.
    Do not persist transformed series in this notebook; this is just for counting.
    """
    # Ensure string-like behavior but keep NA
    s2 = s.astype("string")
    # Normalize common missing strings if they exist
    s2 = s2.replace(["", "NA", "N/A", "None", "null", "NULL"], pd.NA)
    return s2.fillna(MISSING_TOKEN)


def compute_rare_categories(s_norm: pd.Series, min_count: int, min_fraction: float) -> list[str]:
    """
    Determine which categories are rare using a mixed absolute + relative threshold.
    Excludes MISSING_TOKEN from being considered rare by design.
    """
    # Exclude missing token from rarity logic
    s_nomiss = s_norm[s_norm != MISSING_TOKEN]
    counts = s_nomiss.value_counts(dropna=False)

    threshold = max(min_count, int(np.ceil(len(s_norm) * min_fraction)))
    rare = counts[counts < threshold].index.astype(str).tolist()
    return rare


def choose_encoding_strategy(cardinality: int) -> str:
    """
    Determine encoding plan based on cardinality tiers.
    """
    if cardinality <= ONE_HOT_MAX:
        return "one_hot"
    if cardinality <= TARGET_MEAN_MAX:
        return "target_mean"
    return "count_frequency"


# --- Build categorical specs ---
feature_spec.setdefault("features", {}).setdefault("categorical", {})
feature_spec["features"].setdefault("categorical_defaults", cat_defaults)

categorical_summary_rows = []

for col in cat_cols_present:
    s_norm = normalize_missing_for_counts(X[col])

    cardinality = int(s_norm.nunique(dropna=False))
    missing_rate = float((s_norm == MISSING_TOKEN).mean())

    rare_categories = compute_rare_categories(
        s_norm, min_count=MIN_COUNT, min_fraction=MIN_FRACTION
    )

    encoding_plan = choose_encoding_strategy(cardinality)

    feature_spec["features"]["categorical"][col] = {
        "cardinality": cardinality,
        "missing_rate": missing_rate,
        "missing_token": MISSING_TOKEN,
        "rare_token": RARE_TOKEN,
        "unknown_token": UNKNOWN_TOKEN,
        "rare_threshold": {"min_count": MIN_COUNT, "min_fraction": MIN_FRACTION},
        "rare_categories_count": int(len(rare_categories)),
        # NOTE: Do not store the full rare category list unless you want a large spec.
        # If you do want it, store it in a separate artifact (feature_profile.json).
        "encoding_strategy": encoding_plan,
    }

    categorical_summary_rows.append(
        {
            "feature": col,
            "cardinality": cardinality,
            "missing_rate": missing_rate,
            "rare_count": len(rare_categories),
            "encoding_strategy": encoding_plan,
        }
    )

cat_summary_df = pd.DataFrame(categorical_summary_rows).sort_values(
    by=["encoding_strategy", "cardinality"], ascending=[True, False]
)

print("Categorical feature plan (design-only):")
display(cat_summary_df.head(25))

Categorical feature plan (design-only):


,feature,cardinality,missing_rate,rare_count,encoding_strategy
6,zip_code,957,4.440040e-07,928,count_frequency
1,grade,7,0.000000e+00,0,one_hot
4,home_ownership,6,0.000000e+00,3,one_hot
0,term,2,0.000000e+00,0,one_hot
8,application_type,2,0.000000e+00,0,one_hot
7,addr_state,51,0.000000e+00,16,target_mean
2,sub_grade,35,0.000000e+00,9,target_mean
5,purpose,14,0.000000e+00,3,target_mean
3,emp_length,12,6.496666e-02,0,target_mean


# 7. Numerical Feature Engineering

In [30]:
num_cols = eda_summary["numerical_cols"]
num_cols_present = [c for c in num_cols if c in X.columns]

# Ensure spec structures exist
feature_spec.setdefault("features", {}).setdefault("numerical", {})
feature_spec.setdefault("numerical_transform_rules", {})

# Global numerical transform rule (define once)
# Choose ONE convention and stick to it:
# - If you plan to use np.log1p(x + shift): keep function=log1p and applies_to="x + shift_for_transformation"
# - If you plan to use np.log(x + shift): set function="log" and applies_to="x + shift_for_transformation"
feature_spec["numerical_transform_rules"].setdefault(
    "log", {"function": "log1p", "applies_to": "x + shift_for_transformation", "clip_min": 0.0}
)

# Global default: keep original numeric features
feature_spec.setdefault("preserve_original_numerical", True)

# Parameters controlling transform selection
SKEW_THRESHOLD = float(feature_spec.get("numerical_strategy_rules", {}).get("skew_threshold", 1.0))
feature_spec["numerical_strategy_rules"] = {"skew_threshold": SKEW_THRESHOLD}
numerical_summary_rows = []

for col in num_cols_present:
    s = pd.to_numeric(X[col], errors="coerce")

    # Stats for decision-making only
    min_val = float(np.nanmin(s.values)) if np.isfinite(np.nanmin(s.values)) else np.nan
    skew_val = float(s.skew(skipna=True)) if s.notna().sum() > 0 else np.nan
    missing_rate = float(s.isna().mean())

    planned_transform = "none"
    shift = 0.0

    # Decide transform based on skewness
    if np.isfinite(skew_val) and abs(skew_val) > SKEW_THRESHOLD:
        planned_transform = "log"
        # shift required if values can be <= 0
        if np.isfinite(min_val) and min_val <= 0:
            shift = float(abs(min_val) + 1e-6)

    feature_spec["features"]["numerical"][col] = {
        "missing_rate": missing_rate,
        "skewness": skew_val,
        "min_value": min_val,
        "planned_transformation": planned_transform,
        "shift_for_transformation": shift,
    }

    numerical_summary_rows.append(
        {
            "feature": col,
            "missing_rate": missing_rate,
            "skewness": skew_val,
            "min_value": min_val,
            "planned_transformation": planned_transform,
            "shift_for_transformation": shift,
        }
    )

num_summary_df = pd.DataFrame(numerical_summary_rows).sort_values(
    by=["planned_transformation", "skewness"], ascending=[True, False]
)

print("Numerical feature plan (design-only):")
display(num_summary_df.head(25))

with open(FEATURE_SPEC_PATH, "w") as f:
    json.dump(feature_spec, f, indent=4)

Numerical feature plan (design-only):


,feature,missing_rate,skewness,min_value,planned_transformation,shift_for_transformation
15,tot_coll_amt,0.031203,850.514888,0.00,log,0.000001
4,annual_inc,0.000002,494.044495,0.00,log,0.000001
48,num_tl_120dpd_2m,0.068025,55.764378,0.00,log,0.000001
27,total_rev_hi_lim,0.031203,32.629179,0.00,log,0.000001
54,tax_liens,0.000047,32.138175,0.00,log,0.000001
5,dti,0.000756,29.288637,-1.00,log,1.000001
13,annual_inc_joint,0.946890,21.823435,5693.51,log,0.000000
66,sec_app_chargeoff_within_12_mths,0.952478,20.358465,0.00,log,0.000001
49,num_tl_90g_dpd_24m,0.031203,14.914148,0.00,log,0.000001
25,max_bal_bc,0.384225,13.743832,0.00,log,0.000001


# 8. Candidate Engineered Features

- Enumerate candidate engineered features
- Identify source columns for each candidate
- Confirm availability at prediction time
- Assess leakage risk
- Assess interpretability and stability
- Decision recorded for each:
    - include in v1
    - exclude
    - defer to v2
- Rationale documented

| Candidate Feature            | Source Columns              | Description                                           | Inference-Time Available        | Leakage Risk | Decision | Rationale                       |
| ---------------------------- | --------------------------- | ----------------------------------------------------- | ------------------------------- | ------------ | -------- | ------------------------------- |
| `loan_age_months`            | `issue_d`                   | Months since loan issuance relative to reference date | Yes (with fixed reference date) | Low          | Include  | Interpretable temporal signal   |
| `installment_to_income`      | `installment`, `annual_inc` | Monthly payment burden ratio                          | Yes                             | Low          | Include  | Common affordability indicator  |
| `recent_credit_inquiries_6m` | `inq_last_6mths`            | Number of recent credit inquiries                     | Yes                             | Low          | Exclude  | Widely used credit risk feature |

These candidate features were evaluated for feasibility (derived from source columns), leakage risk (are computable at time of origination and do not consider future information), and interpretability. All of these are suitable for v1 and will be included to the feature specification.

In [ ]:
feature_spec["features"]["numerical"]["loan_age_months"] = {
    "source": ["issue_d"],
    "type": "derived",
    "derivation": {
        "method": "months_since",
        "reference_date": "prediction_time",
        "rounding": "floor",
    },
    "availability": "inference_time",
    "planned_transformation": "none",
    "notes": "Computed as full months elapsed since loan issue date",
}

feature_spec["features"]["numerical"]["installment_to_income"] = {
    "source": ["installment", "annual_inc"],
    "type": "derived",
    "derivation": {
        "method": "ratio",
        "numerator": "installment",
        "denominator": "annual_inc",
        "zero_division": "null",
        "clip": {"min": 0.0, "max": None},
    },
    "availability": "inference_time",
    "planned_transformation": "none",
    "notes": "Monthly payment burden relative to reported annual income",
}

# feature_spec["features"]["numerical"]["recent_credit_inquiries_6m"] = {
#     "source": ["inq_last_6mths"],
#     "type": "rename",
#     "derivation": {"method": "identity"},
#     "availability": "inference_time",
#     "planned_transformation": "none",
#     "notes": "Renamed for semantic clarity; no transformation applied",
# }

# 9. Feature Interactions

| Interaction                                          | Source Features                            | Evaluation                          | Decision     | Rationale                                                   |
| ---------------------------------------------------- | ------------------------------------------ | ----------------------------------- | ------------ | ----------------------------------------------------------- |
| `installment_to_income × loan_age_months`            | `installment_to_income`, `loan_age_months` | Interpretable but adds nonlinearity | Exclude (v1) | Captured by nonlinear models without explicit interaction   |
| `recent_credit_inquiries_6m × installment_to_income` | `inq_last_6mths`, `installment_to_income`  | Potential risk signal               | Exclude (v1) | Increases feature complexity without clear benefit          |
| `grade × installment_to_income`                      | `grade`, `installment_to_income`           | Domain-plausible                    | Exclude (v1) | Requires careful encoding; defer until baseline established |


Decision:
- No explicit interactions features to be included in v1.
- Nonlinear relationships are delegated to the modeling stage.

Rationale:
- Preserve interpretability.
- Avoid combinatorial expansion.
- Maintain clean FE/modeling boundary.
- Establish a strong baseline before adding complexity.

# 10. Missing Value Strategy (Design Only)

Strategy:
- Categorical policy:
    - Missing -> __MISSING__
    - Unknown/rate -> __OTHER__
- Numerical policy:
    - Imputer: median
    - Fit on training data only
    - Add missing indicators
- Datetime-derived policy
    - Missing raw datetime -> missing derived fields
    - Derived fields filled with sentinel -1
    - Add missing indicator for raw datetime

In [32]:
import pandas as pd

num_cols = [c for c in eda_summary["numerical_cols"] if c in X.columns]
cat_cols = [c for c in eda_summary["categorical_cols"] if c in X.columns]
dt_cols = [c for c in eda_summary["datetime_cols"] if c in X.columns]


def missing_rate(series):
    return float(series.isna().mean())


summary = []
for c in num_cols:
    summary.append(
        {
            "col": c,
            "family": "numerical",
            "missing_rate": missing_rate(pd.to_numeric(X[c], errors="coerce")),
        }
    )
for c in cat_cols:
    summary.append({"col": c, "family": "categorical", "missing_rate": missing_rate(X[c])})
for c in dt_cols:
    parsed = pd.to_datetime(X[c], errors="coerce")
    summary.append({"col": c, "family": "datetime", "missing_rate": missing_rate(parsed)})

miss_df = pd.DataFrame(summary).sort_values(["family", "missing_rate"], ascending=[True, False])
display(miss_df.head(30))

,col,family,missing_rate
70,emp_length,categorical,6.496666e-02
73,zip_code,categorical,4.440040e-07
67,term,categorical,0.000000e+00
68,grade,categorical,0.000000e+00
69,sub_grade,categorical,0.000000e+00
71,home_ownership,categorical,0.000000e+00
72,purpose,categorical,0.000000e+00
74,addr_state,categorical,0.000000e+00
75,application_type,categorical,0.000000e+00
76,issue_d,datetime,0.000000e+00


Missing value handling is defined in the feature spec and deferred to the build/training pipeline. Categorical missingness is tokenized; numerical missingness is imputed with median (fit on train only) with missing indicators; datetime missingness is handled via sentinel fill for derived components with a raw missing indicator.

# 11. Feature Specification Output

In [33]:
import hashlib
import json
from datetime import datetime, timezone
from pathlib import Path

# -----------------------------
# Paths
# -----------------------------

ARTIFACTS_DIR = Path("../data/_artifacts_preview")
FEATURE_SPEC_PATH = ARTIFACTS_DIR / "feature_spec_v1.json"

# -----------------------------
# Validate spec completeness (fail fast)
# -----------------------------
required_top_level = ["version", "source", "target", "features", "freeze"]
missing_top = [k for k in required_top_level if k not in feature_spec]
if missing_top:
    raise KeyError(f"feature_spec missing top-level keys: {missing_top}")

# Exclusions must exist from Step 3
if "exclusions" not in feature_spec:
    raise KeyError(
        "feature_spec missing 'exclusions'. Ensure Step 3 populated feature_spec['exclusions']."
    )

for k in ["target", "excluded_feature_cols", "by_reason"]:
    if k not in feature_spec["exclusions"]:
        raise KeyError(f"feature_spec['exclusions'] missing '{k}' (Step 3 should set it).")

# Required feature blocks (adjust only if intentionally omitted)
required_feature_blocks = ["datetime", "categorical", "numerical"]
missing_blocks = [b for b in required_feature_blocks if b not in feature_spec["features"]]
if missing_blocks:
    raise KeyError(f"feature_spec['features'] missing required blocks: {missing_blocks}")

# Target constraints sanity
TARGET = feature_spec["target"].get("name")
if not TARGET or TARGET not in df.columns:
    raise KeyError(f"Target '{TARGET}' not found in dataset columns.")

allowed_values = feature_spec["target"].get("allowed_values", [0, 1])
if allowed_values != [0, 1]:
    raise ValueError(f"Target allowed_values must be [0, 1]. Got: {allowed_values}")

if feature_spec["target"].get("missing_allowed") is not False:
    raise ValueError("Target missing_allowed should be False.")

# Ensure target is not in excluded_feature_cols
excluded_feature_cols = feature_spec["exclusions"]["excluded_feature_cols"]
if TARGET in excluded_feature_cols:
    raise ValueError(
        "Target is in excluded_feature_cols. Target must be excluded separately, not in feature exclusions."
    )

# -----------------------------
# Freeze metadata
# -----------------------------
# created_at (UTC)
feature_spec["freeze"]["created_at"] = datetime.now(timezone.utc).isoformat()

# target content hash (sanity check; changes if dataset rows change)
y = df[TARGET]
target_hash_md5 = hashlib.md5(y.to_numpy().tobytes()).hexdigest()
feature_spec["freeze"]["target_hash_md5"] = target_hash_md5

# spec hash (avoid hashing itself)
feature_spec["freeze"]["spec_hash_md5"] = None
spec_json_for_hash = json.dumps(feature_spec, sort_keys=True).encode("utf-8")
spec_hash_md5 = hashlib.md5(spec_json_for_hash).hexdigest()
feature_spec["freeze"]["spec_hash_md5"] = spec_hash_md5

# -----------------------------
# Write spec (ONCE)
# -----------------------------
with open(FEATURE_SPEC_PATH, "w") as f:
    json.dump(feature_spec, f, indent=2, sort_keys=True)

print(f"Wrote feature spec: {FEATURE_SPEC_PATH}")
print(f"spec_hash_md5:   {spec_hash_md5}")
print(f"target_hash_md5: {target_hash_md5}")

# -----------------------------
# Read-back validation
# -----------------------------
with open(FEATURE_SPEC_PATH, "r") as f:
    spec_check = json.load(f)

assert spec_check["freeze"]["spec_hash_md5"] == spec_hash_md5, "Spec hash mismatch after write"
assert spec_check["freeze"]["target_hash_md5"] == target_hash_md5, (
    "Target hash mismatch after write"
)

print("Read-back validation passed.")

Wrote feature spec: ../data/_artifacts_preview/feature_spec_v1.json
spec_hash_md5:   4510e26a43f9a028fb2aa5aa8c65c29d
target_hash_md5: 2cffb60f71c9c857a47da555b72333dd
Read-back validation passed.


In [ ]:
import json
from pathlib import Path

import numpy as np
import pandas as pd

# --- paths (repo-relative by default; works on Mac + in repo) ---
# PROJECT_ROOT = Path.cwd()  # run notebook from repo root
ARTIFACTS_DIR = Path("../data/_artifacts_preview")
FEATURE_SPEC_PATH = ARTIFACTS_DIR / "feature_spec_v1.json"
EDA_CLEAN_PATH = ARTIFACTS_DIR / "eda_cleaned.parquet"  # adjust if different
OUT_PATH = ARTIFACTS_DIR / "engineered_features.parquet"

ARTIFACTS_DIR.mkdir(parents=True, exist_ok=True)
if not FEATURE_SPEC_PATH.exists():
    raise FileNotFoundError(f"Missing: {FEATURE_SPEC_PATH}")

# --- load spec ---
spec = json.loads(FEATURE_SPEC_PATH.read_text())

# --- load starting point ---
if not EDA_CLEAN_PATH.exists():
    raise FileNotFoundError(f"Missing: {EDA_CLEAN_PATH}")
df = pd.read_parquet(EDA_CLEAN_PATH).copy()

# --- extract expected feature columns from spec ---
num_cols = list(spec.get("features", {}).get("numerical", {}).keys())
cat_cols = list(spec.get("features", {}).get("categorical", {}).keys())
dt_cols = list(spec.get("features", {}).get("datetime", {}).keys())
expected_features = num_cols + cat_cols + dt_cols

# 1) loan_age_months
if "loan_age_months" in expected_features and "loan_age_months" not in df.columns:
    issue_dates = pd.to_datetime(df["issue_d"], errors="coerce")
    reference_date = pd.Timestamp.now()
    df["loan_age_months"] = (
        (reference_date.year - issue_dates.dt.year) * 12
        + (reference_date.month - issue_dates.dt.month)
    ).clip(lower=0)

# 2) installment_to_income
if "installment_to_income" in expected_features and "installment_to_income" not in df.columns:
    df["installment_to_income"] = (
        (df["installment"] / df["annual_inc"]).replace([np.inf, -np.inf], np.nan).clip(lower=0)
    )

# 3) recent_credit_inquiries_6m
# if "recent_credit_inquiries_6m" in expected_features and "recent_credit_inquiries_6m" not in df.columns:
#     if "inq_last_6mths" in df.columns:
#         df["recent_credit_inquiries_6m"] = df["inq_last_6mths"]
#     else:
#         # If source column is missing, skip this feature or fill with null
#         print(f"WARNING: Cannot create 'recent_credit_inquiries_6m': source column 'inq_last_6mths' not found. Filling with NaN.")
#         df["recent_credit_inquiries_6m"] = np.nan

# --- enforce contract: all spec features exist ---
missing = [c for c in expected_features if c not in df.columns]
if missing:
    raise AssertionError(
        "Feature-spec mismatch: these features are in feature_spec_v1.json but not in the dataframe:\n"
        + "\n".join(missing)
    )

# --- write out parquet (features + optional target) ---
target_col = spec.get("target", {}).get("name")

cols_out = expected_features + ([target_col] if target_col and target_col in df.columns else [])
df_out = df.loc[:, cols_out].copy()

df_out.to_parquet(OUT_PATH, index=False)

print("✔ wrote:", OUT_PATH)
print("  rows:", len(df_out), "cols:", len(df_out.columns))
print("  columns:", list(df_out.columns))

✔ wrote: ../data/_artifacts_preview/engineered_features.parquet
  rows: 2252232 cols: 81
  columns: ['acc_open_past_24mths', 'all_util', 'annual_inc', 'annual_inc_joint', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util', 'delinq_2yrs', 'dti', 'dti_joint', 'fico_range_high', 'fico_range_low', 'funded_amnt', 'il_util', 'inq_fi', 'installment', 'installment_to_income', 'int_rate', 'loan_age_months', 'loan_amnt', 'max_bal_bc', 'mo_sin_old_il_acct', 'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl', 'mort_acc', 'num_actv_bc_tl', 'num_actv_rev_tl', 'num_bc_sats', 'num_bc_tl', 'num_il_tl', 'num_op_rev_tl', 'num_rev_accts', 'num_rev_tl_bal_gt_0', 'num_sats', 'num_tl_120dpd_2m', 'num_tl_90g_dpd_24m', 'num_tl_op_past_12m', 'open_acc', 'open_acc_6m', 'open_act_il', 'open_il_12m', 'open_il_24m', 'open_rv_12m', 'open_rv_24m', 'pct_tl_nvr_dlq', 'percent_bc_gt_75', 'pub_rec', 'pub_rec_bankruptcies', 'recent_credit_inquiries_6m', 'revol_bal', 'revol_bal_joint', 'sec_app_chargeoff_within_12_m